# Harmony Regression 
This notebook provides condensed examples of using Harmony to perform specific tasks.  For more a general introduction and tutorial, see [Harmony API Introduction](https://harmony.earthdata.nasa.gov/notebook-example.html) or any of the example notebooks provided in the [Harmony repository](https://github.com/nasa/harmony). [Helper functions](./notebook_helpers) are used for making the calls found in this notebook.

## Prerequisites

 The dependencies for this notebook are listed in the [environment.yaml](./environment.yaml). To test or install locally, create the papermill-<IMAGE> environment used in the automated regression testing suite: 

`conda env create -f ./environment.yaml && conda activate papermill-<IMAGE>`

Also ensure you have a `.netrc` file located in the `test` directory of this repository.

## Set harmony_host_url and import libraries 

In [ ]:
# harmony_host_url = 'https://harmony.sit.earthdata.nasa.gov'
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'
# harmony_host_url = 'https://harmony.earthdata.nasa.gov'

In [ ]:
from notebook_helpers import (
    get,
    post,
    show,
    show_async_condensed,
    print_async_status,
)

## Example Data

Harmony has produced example collections with artificial data but realistic structure to allow testing our services.  We have L3 and L2 NetCDF4 collections, and a shapefile collection.

In [ ]:
coverages_root = '{root}/{collection}/ogc-api-coverages/1.0.0/collections/{variable}/coverage/rangeset'
edr_root = '{root}/ogc-api-edr/1.1.0/collections/{collection}/cube'

if harmony_host_url == 'https://harmony.earthdata.nasa.gov':
    is_not_prod = False
else:
    is_not_prod = True

if is_not_prod:
    l3_collection = 'C1234088182-EEDTEST'
    l2_collection = 'C1233860183-EEDTEST'
    shapefile_collection = 'C1234530533-EEDTEST'
    l2ss_collection = 'C1234208438-POCLOUD'
    var_collection = 'C1234714698-EEDTEST'
    download_collection = 'C1238392622-EEDTEST'
    download_link_substring = 'harmony-uat-eedtest-data'
else:
    l3_collection = 'C1756916832-XYZ_PROV'
    l2_collection = 'C1756916832-XYZ_PROV'
    l2ss_collection = 'C1940473819-POCLOUD'
    download_collection = 'C1996546500-GHRC_DAAC'
    download_link_substring = 'data.ghrc.earthdata.nasa.gov'

example_vars = ['red_var', 'green_var', 'blue_var', 'alpha_var']

## Set up Harmony Client

This is for use with requests, which will eventually replace some of the functionality in `notebook_helpers`.

In [ ]:
from harmony import Client, Collection, Environment, Request

host_environment = {
    'http://localhost:3000': Environment.LOCAL,
    'https://harmony.sit.earthdata.nasa.gov': Environment.SIT,
    'https://harmony.uat.earthdata.nasa.gov': Environment.UAT,
    'https://harmony.earthdata.nasa.gov': Environment.PROD,
}


harmony_environment = host_environment.get(harmony_host_url)

if harmony_environment is not None:
    harmony_client = Client(env=harmony_environment)

### Sample OGC Coverages Sync Request - Bounding Box and Temporal Subsetting with Reformatting

harmony/service-example only working in UAT due to Harmony-600.

In [ ]:
if is_not_prod:
    # By default, this reformats to tiff
    params = {
        'subset': [
            'lon(-20:90)',
            'lat(0:60)',
            'time("2020-01-15T00:00:00Z":"2020-01-15T01:00:00Z")',
        ]
    }

    response = get(
        coverages_root.format(
            root=harmony_host_url, collection=l3_collection, variable='all'
        ),
        params=params,
    )

    show(response, example_vars)

In [ ]:
if is_not_prod:
    response = get(
        coverages_root.format(
            root=harmony_host_url, collection=l3_collection, variable='green_var'
        ),
        params=params,
    )

    show(response, color_index=1)

### Sample OGC EDR Sync Request - Bounding Box and Temporal Subsetting with Reformatting

harmony/service-example only working in UAT due to Harmony-600.

In [ ]:
if is_not_prod:
    # By default, this reformats to tiff
    response = get(
        edr_root.format(root=harmony_host_url, collection=l3_collection),
        params={
            'bbox': '-20,0,90,60',
            'datetime': '2020-01-15T00:00:00Z/2020-01-15T01:00:00Z',
        },
    )

    show(response, example_vars)

In [ ]:
if is_not_prod:
    response = get(
        edr_root.format(root=harmony_host_url, collection=l3_collection),
        params={
            'bbox': '-20,0,90,60',
            'datetime': '2020-01-15T00:00:00Z/2020-01-15T01:00:00Z',
            'parameter-name': 'green_var',
        },
    )

    show(response, color_index=1)

## Asynchronous Requests

harmony/service-example only working in UAT due to Harmony-600.

In [ ]:
if is_not_prod:
    response = get(
        coverages_root.format(
            root=harmony_host_url, collection=l3_collection, variable='all'
        ),
        params={
            'subset': [
                'lon(-20:90)',
                'lat(0:60)',
                'time("2020-01-01T00:00:00Z":"2020-01-05T01:00:00Z")',
            ]
        },
    )
    show_async_condensed(response)

## Cancel Async Requests

harmony/service-example only working in UAT due to Harmony-600.

In [ ]:
if is_not_prod:
    # Add 3 requests
    response1 = get(
        coverages_root.format(
            root=harmony_host_url, collection=l3_collection, variable='all'
        ),
        params={'format': 'image/tiff', 'maxResults': '20'},
    )

    response2 = get(
        coverages_root.format(
            root=harmony_host_url, collection=l3_collection, variable='all'
        ),
        params={'format': 'image/tiff', 'maxResults': '20'},
    )

    response3 = get(
        coverages_root.format(
            root=harmony_host_url, collection=l3_collection, variable='all'
        ),
        params={'format': 'image/tiff', 'maxResults': '20'},
    )

    # List the requests

    jobs_root = '{root}/jobs'
    my_jobs = jobs_root.format(root=harmony_host_url)
    response = get(my_jobs, params={'page': '1', 'limit': '10'})
    body = response.json()

    for job in body['jobs']:
        print_async_status(job)

    # Cancel one
    my_jobs_cancel_root = my_jobs + '/{job_id}/cancel'
    response = post(my_jobs_cancel_root.format(job_id=response3.json()['jobID']))

    print_async_status(response.json())

    assert response.json()['status'] == 'canceled'

# Basic Tests with Backend Services

### Data Services:

The following services are tested in service-specific Jupyter notebooks:

* Geoloco
* Harmony Browse Image Generator (HyBIG)
* Harmony GDAL Adapter (HGA)
* Harmony OPeNDAP SubSetter (HOSS)
* Harmony Regridding Service
* Swath Projector
* Trajectory Subsetter
* Variable Subsetter

### PODAAC L2 Subsetter

#### PODAAC L2 Subsetter (Sync)

In [ ]:
response = get(
    coverages_root.format(
        root=harmony_host_url, collection=l2ss_collection, variable='all'
    ),
    params={
        'maxResults': '1',
        'subset': [
            'lon(-160:160)',
            'lat(-80:80)',
            'time("2012-03-03T12:17:00":"2012-03-03T12:18:00")',
        ],
    },
)

if is_not_prod:
    show(response, ['ssha'])
else:
    show(response, ['sea_surface_temperature'])

#### PODAAC L2 Subsetter (Async)

In [ ]:
response = get(
    coverages_root.format(
        root=harmony_host_url, collection=l2ss_collection, variable='all'
    ),
    params={
        'maxResults': '3',
        'format': 'application/x-netcdf4',
        'subset': [
            'lon(-160:160)',
            'lat(-80:80)',
            'time("2012-03-03T00:00:00Z":"2012-03-03T02:59:59Z")',
        ],
    },
)

if is_not_prod:
    show_async_condensed(response, ['ssha'])
else:
    show_async_condensed(response, ['sea_surface_temperature'])

## harmony/download service

[HARMONY-1649](https://bugs.earthdata.nasa.gov/browse/HARMONY-1649) added the `harmony/download` service, which will be called when no transformation is specified in the request parameters (e.g., subsetting, reformatting, reprojections).

The request below establishes that the service is used, rather than one of the other Harmony backend services. Because there is nothing in the `/jobs` output indicating the service that was used, this test asserts that at least one of the output links from Harmony contains a substring that is tied to the original source data.

In [ ]:
download_service_request = Request(
    collection=Collection(download_collection), max_results=1
)

download_service_job_id = harmony_client.submit(download_service_request)
download_service_json = harmony_client.result_json(download_service_job_id)

assert any(
    download_link_substring in link['href']
    for link in download_service_json.get('links')
    if link['rel'] == 'data'
), 'Download service did not return expected link.'

_Minimizing time to science_